In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, date, time
import matplotlib.pyplot as plt
%matplotlib inline
#plt.rcParams['figure.dpi'] = 300
%config InlineBackend.figure_format='svg'
import seaborn as sns
train_data=pd.read_csv(r'C:\Users\milk\Desktop\Project 2\train.csv')
test_data=pd.read_csv(r'C:\Users\milk\Desktop\Project 2\test.csv')
train_data
test_data

,year,month,day,hour,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,2014,1,1,0,-20.0,7.0,1014.0,NW,143.48,0,0
1,2014,1,1,1,-20.0,7.0,1013.0,NW,147.50,0,0
2,2014,1,1,2,-20.0,6.0,1013.0,NW,151.52,0,0
3,2014,1,1,3,-20.0,6.0,1013.0,NW,153.31,0,0
4,2014,1,1,4,-18.0,3.0,1012.0,cv,0.89,0,0
...,...,...,...,...,...,...,...,...,...,...,...
8755,2014,12,31,19,NaN,-2.0,1034.0,NW,231.97,0,0
8756,2014,12,31,20,-22.0,-3.0,1034.0,NW,237.78,0,0
8757,2014,12,31,21,-22.0,NaN,NaN,NW,242.70,0,0
8758,2014,12,31,22,NaN,-4.0,1034.0,NW,246.72,0,0


In [2]:
col_names=(test_data).columns

In [3]:
print(train_data.isnull().sum())  

year         0
month        0
day          0
hour         0
pm2.5     1968
DEWP      5259
TEMP     10519
PRES     14025
cbwd         0
Iws          0
Is           0
Ir           0
dtype: int64


In [4]:
print(test_data.isnull().sum())  

year        0
month       0
day         0
hour        0
DEWP      876
TEMP     1752
PRES      876
cbwd        0
Iws         0
Is          0
Ir          0
dtype: int64


In [5]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train_data['cbwd'])
train_data['cbwd']= le.transform(train_data['cbwd'])
test_data['cbwd']= le.transform(test_data['cbwd'])

In [6]:
train_y=train_data['pm2.5']
train_x=train_data.drop('pm2.5',1)
train_y = train_y.fillna(np.mean(train_y))

In [7]:
print(train_x.isnull().sum())  

year         0
month        0
day          0
hour         0
DEWP      5259
TEMP     10519
PRES     14025
cbwd         0
Iws          0
Is           0
Ir           0
dtype: int64


In [8]:
#missing value 使用前向填充法
train_x[['DEWP','TEMP','PRES']]=train_x[['DEWP','TEMP','PRES']].fillna(method = 'ffill')
test_data[['DEWP','TEMP','PRES']]=test_data[['DEWP','TEMP','PRES']].fillna(method = 'ffill')

In [9]:
print(train_x.isnull().sum())  
print(test_data.isnull().sum())  

year     0
month    0
day      0
hour     0
DEWP     0
TEMP     0
PRES     0
cbwd     0
Iws      0
Is       0
Ir       0
dtype: int64
year     0
month    0
day      0
hour     0
DEWP     0
TEMP     0
PRES     0
cbwd     0
Iws      0
Is       0
Ir       0
dtype: int64


In [12]:
#納入季節
#1-Spring(3 4 5) 2-Summer(6 7 8) 3-Fall(9 10 11) 4-Winter(12 1 2)
def getseason(y):
    if 3 <= y <= 5 :
        Season = 1
    elif 6 <= y <= 8 :
        Season = 2
    elif 9 <= y <= 11 :
        Season = 3
    else :
        Season = 4
    return Season
train_x['season'] = train_x['month'].apply(getseason)
test_data['season'] = test_data['month'].apply(getseason)

In [13]:
#露點與溫度差
train_x['diff'] = train_x['DEWP']-train_x['TEMP']
test_data['diff'] = test_data['DEWP']-test_data['TEMP']

In [15]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')

In [16]:
imp.fit(train_x)
train_x = imp.transform(train_x)
#train_x=pd.DataFrame(train_x,columns=col_names)
train_x

array([[ 2.010e+03,  1.000e+00,  1.000e+00, ...,  0.000e+00,  4.000e+00,
        -1.000e+01],
       [ 2.010e+03,  1.000e+00,  1.000e+00, ...,  0.000e+00,  4.000e+00,
        -9.000e+00],
       [ 2.010e+03,  1.000e+00,  1.000e+00, ...,  0.000e+00,  4.000e+00,
        -1.000e+01],
       ...,
       [ 2.013e+03,  1.200e+01,  3.100e+01, ...,  0.000e+00,  4.000e+00,
        -2.800e+01],
       [ 2.013e+03,  1.200e+01,  3.100e+01, ...,  0.000e+00,  4.000e+00,
        -2.800e+01],
       [ 2.013e+03,  1.200e+01,  3.100e+01, ...,  0.000e+00,  4.000e+00,
        -2.700e+01]])

In [18]:
train_y = train_y.fillna(np.mean(train_y))
#train__y=pd.DataFrame(train_y)
train_y

0        98.843153
1        98.843153
2        98.843153
3        98.843153
4        98.843153
           ...    
35059    22.000000
35060    18.000000
35061    23.000000
35062    20.000000
35063    23.000000
Name: pm2.5, Length: 35064, dtype: float64

In [19]:
i=10
train_y[i]

98.84315325114818

In [27]:
from sklearn.ensemble import RandomForestRegressor

In [28]:
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

In [29]:
rf.fit(train_x, train_y)

RandomForestRegressor(n_estimators=1000, random_state=42)

In [31]:
predictions = rf.predict(test_data)

In [34]:
len(predictions)

8760

In [36]:
ID=range(1,8761)
ans=pd.DataFrame({'ID':ID,'Predicted':predictions})
ans

,ID,Predicted
0,1,27.649275
1,2,27.346804
2,3,26.029804
3,4,23.741647
4,5,95.958529
...,...,...
8755,8756,17.166902
8756,8757,17.358118
8757,8758,16.951745
8758,8759,17.203745


In [37]:
ans.to_csv(r'C:\Users\milk\Desktop\project 2\project2_RF.csv',index=False)